In [1]:
import os, sys
import cv2
import time
import pickle
import shutil
import argparse
import traceback
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
from code.utils import *

# Create dataframes for interested independent variables
* Interested independent variables: smile duration, smile timing, intensity, facial asymmetry
* Integrate these values from all videos into several dataframes

In [2]:
## video subfolders
freq = 5
category = 'pd'
if category == 'pd':
    video_folders = ['pd_1-100', 'pd_101-200', 'pd_201-300', 'pd_301-400', 'pd_401-500',
                 'pd_501-600', 'pd_601-700', 'pd_701-800', 'pd_801-897']
elif category == 'fin':
    video_folders = ['video_1-100', 'video_101-200', 'video_201-300', 'video_301-350', 'video_351-400', 'video_401-500', \
                     'video_501-580', 'video_581-660', 'video_661-740', 'video_741-820', 'video_821-900']
video_paths = [f'{category}/{x}' for x in video_folders]
video_lookup = dict.fromkeys(video_folders)
for i, k in enumerate(video_lookup.keys()):
    video_lookup[k] = [os.path.splitext(x)[0] for x in os.listdir(f'/home/xyubl/Downloads/{k}')]
video_lookup_inv = {}
for k, v in video_lookup.items():
    for vid in v:
        video_lookup_inv[vid] = k

In [3]:
## utility functions
def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

def get_land_diff(lands):
    n = len(lands)
    diffs = np.diff(lands, axis=0) # (n-1, 68, 2)
    tmp = np.vstack(diffs) # ((n-1)*68, 2)
    norms = np.linalg.norm(tmp, axis=1).reshape(68, n-1)
    med = np.median(np.mean(norms, axis=0))
    return med

In [4]:
## calculate smile duration, timing and filter info for all videos
all_duration, all_timing_3, all_intensity, all_asymmetry, all_filter = {}, {}, {}, {}, {}
for video_folder, video_path in zip(video_folders, video_paths):
    dur_df = pd.DataFrame(columns=['Video_ID', 'showup', 'smile_duration', 'duration', 'smile_proportion'])
    tim3_df = pd.DataFrame(columns=['Video_ID', 'showup', 'beginning', 'middle', 'end'])
    inten_df = pd.DataFrame(columns=['Video_ID', 'showup', 'Intensity'])
    asym_df = pd.DataFrame(columns=['Video_ID', 'showup', 'asymmetry'])
    filter_df = pd.DataFrame(columns=['Video_ID', 'showup', 'video_duration', 'showup_proportion', 'frame_land_diff', 'align_land_diff'])
    videos = os.listdir(f'{video_path}/freq1_scores')
    for video in tqdm(videos):
        # video info
        cap = cv2.VideoCapture(f'/home/xyubl/Downloads/{video_folder}/{video}.mp4')
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        frame_count = int(cap.get(cv2. CAP_PROP_FRAME_COUNT))
        video_duration = round(frame_count/fps, 2) # seconds
        # whether speaker shows up
        emo_file = f'{video_path}/freq{freq}_scores/{video}/target_emotion_vai.csv'
        asym_file = f'{video_path}/freq{freq}_scores/{video}/target_asymmetry.csv'
        # print(f'Video {video} speaker shows up: {os.path.exists(emo_file)}')
        if os.path.exists(emo_file):
            emo_df = pd.read_csv(emo_file, sep='\t')
            asym_df_ = pd.read_csv(asym_file, sep='\t')
            ## get time and smiling frames
            time = [int(x.split('_')[1].replace('msec', ''))/1000 for x in emo_df.Frame_ID.values.tolist()]
            emo = emo_df.iloc[:,1:9].values.argmax(axis=1)
            smile = (emo == 1).astype(int)
            # get smiling duration
            smile_dur = smile.sum() * fps
            dur = len(smile) * fps
            smile_prop = smile_dur / dur
            # print(fps, smile_dur, dur, smile_prop)
            dur_df.loc[len(dur_df)] = [video, 1, smile_dur, dur, smile_prop]
            ## timing 3
            if len(smile) < 3:
                tim3_df.loc[len(tim3_df)] = [video, 1, 0, 0, 0]
            else:
                chunks = list(split(smile, 3))
                chunk_props = [sum(x)/len(x) for x in chunks]
                tim3_df.loc[len(tim3_df)] = [video, 1] + chunk_props
            ## intensity
            inten_df.loc[len(inten_df)] = [video, 1, emo_df.Intensity.mean()]
            ## asymmetry
            asym_df.loc[len(asym_df)] = [video, 1, asym_df_.asymmetry.mean()]

            ## filter info
            # showup_proportion = len(smile) / frame_count * (fps / freq)
            with open(f'{video_path}/freq1_scores/{video}/frame_info_2d_68pts_encodings.pkl', 'rb') as f:
                tmp = pickle.load(f)
            info = pd.read_csv(f'{video_path}/freq1_scores/{video}/frame_info.csv', sep='\t')
            showup_proportion = len(info) / frame_count * fps
            target_frames = info.loc[info['is_target']==1, 'Frame_ID'].values.tolist()
            frame_lands, align_lands = [], []
            for k in target_frames:
                frame_lands.append(landmarks_to_np(tmp[k]['2d_landmark']))
                img = cv2.imread(f'{video_path}/freq1_aligned/{video}/{k}.jpg')
                face_locations = face_detector(img)
                if len(face_locations) == 1:
                    face_location = face_locations[0]
                    landmark = pose_predictor_68_point(img, face_location)
                    save_2d_landmarks = landmarks_to_np(landmark) 
                    align_lands.append(save_2d_landmarks)
            if len(frame_lands) > 0:
                frame_land_diff = get_land_diff(frame_lands)
            else:
                frame_land_diff = None
            if len(align_lands) > 0:
                align_land_diff = get_land_diff(align_lands)
            else:
                align_land_diff = None
            filter_df.loc[len(filter_df)] = [video, 1, video_duration, showup_proportion, frame_land_diff, align_land_diff]
            # print([video, 1, video_duration, showup_proportion, frame_land_diff, align_land_diff])
        else:
            dur_df.loc[len(dur_df)] = [video, 0, None, None, None]
            tim3_df.loc[len(tim3_df)] = [video, 0, None, None, None]
            filter_df.loc[len(filter_df)] = [video, 0, video_duration, 0, None, None]
            inten_df.loc[len(inten_df)] = [video, 0, None]
            asym_df.loc[len(asym_df)] = [video, 0, None]
    all_duration[video_folder] = dur_df
    all_timing_3[video_folder] = tim3_df
    all_intensity[video_folder] = inten_df
    all_asymmetry[video_folder] = asym_df
    all_filter[video_folder] = filter_df

    print(video_path)

In [5]:
## concatenate all video folders
all_duration_df = pd.concat(all_duration.values()).sort_values(by='Video_ID').reset_index(drop=True)
all_timing_3_df = pd.concat(all_timing_3.values()).sort_values(by='Video_ID').reset_index(drop=True)
all_intensity_df = pd.concat(all_intensity.values()).sort_values(by='Video_ID').reset_index(drop=True)
all_asymmetry_df = pd.concat(all_asymmetry.values()).sort_values(by='Video_ID').reset_index(drop=True)
all_filter_df = pd.concat(all_filter.values()).sort_values(by='Video_ID').reset_index(drop=True)
all_duration_df.shape, all_timing_3_df.shape, all_intensity_df.shape, all_asymmetry_df.shape, all_filter_df.shape

In [6]:
## save dataframes
all_duration_df.to_csv(f'{category}/duration_freq{freq}.csv', sep='\t', index=False)
all_timing_3_df.to_csv(f'{category}/timing_3_freq{freq}.csv', sep='\t', index=False)
all_intensity_df.to_csv(f'{category}/intensity_freq{freq}.csv', sep='\t', index=False)
all_asymmetry_df.to_csv(f'{category}/asymmetry_freq{freq}.csv', sep='\t', index=False)
all_filter_df.to_csv(f'{category}/video_filter.csv', sep='\t', index=False)